## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random
from sklearn import metrics

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [2]:
data= pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
data.head()

,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,...,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,...,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,...,1,1,0,1,0,0,0,1,0,5


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [3]:
q5_cols = [i for i in data.columns if i[:2] == 'Q5']
q5_data = data[q5_cols]
q5_data

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0
2954,0,1,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0


The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [4]:
matrix = squareform(pdist(q5_data, metric = 'hamming'))
matrix_df = pd.DataFrame(matrix)
matrix_df

,0,1,2,3,4,5,6,7,8,9,...,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,...,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
3,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,...,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
4,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,...,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,0.2,0.2,0.3,0.6,0.3,0.3,0.2,0.5,0.4,0.4,...,0.1,0.1,0.4,0.3,0.5,0.0,0.2,0.2,0.4,0.2
2954,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2955,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2956,0.2,0.2,0.1,0.6,0.5,0.3,0.2,0.3,0.6,0.4,...,0.3,0.5,0.4,0.5,0.5,0.4,0.2,0.2,0.0,0.2


Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [5]:
k = 2
init_medoids = np.random.randint(0, 200, k)
init_medoids

kmed = kmedoids(
    matrix, initial_index_medoids=init_medoids, data_type="distance_matrix"
)

kmed.process()

medoid_idxs = kmed.get_medoids()
medoid_idxs

[0, 190]

In [6]:
labels = kmed.predict(matrix)
labels

array([0, 0, 1, ..., 0, 1, 1])

In [7]:
print(metrics.silhouette_score(matrix, labels, metric='euclidean'))

0.18443318045383486


If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial centroids that are far apart. Re-run the previous process, except with a random pair of centroids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [8]:
import sys

In [9]:
index_list = list()
for index, row in matrix_df.iterrows():
    small_list = list()
    for i,v in row.iteritems():
        if v>=0.8:
            small_list.append(index)
            small_list.append(i)
            index_list.append(small_list)
            small_list = list()

index_list

[[0, 245],
 [0, 263],
 [0, 496],
 [0, 752],
 [0, 812],
 [0, 866],
 [0, 878],
 [0, 923],
 [0, 941],
 [0, 1125],
 [0, 1282],
 [0, 1291],
 [0, 1338],
 [0, 1472],
 [0, 1578],
 [0, 1605],
 [0, 1796],
 [0, 2041],
 [0, 2706],
 [0, 2724],
 [0, 2899],
 [1, 245],
 [1, 263],
 [1, 496],
 [1, 752],
 [1, 812],
 [1, 866],
 [1, 878],
 [1, 923],
 [1, 941],
 [1, 1125],
 [1, 1282],
 [1, 1291],
 [1, 1338],
 [1, 1472],
 [1, 1578],
 [1, 1605],
 [1, 1796],
 [1, 2041],
 [1, 2706],
 [1, 2724],
 [1, 2899],
 [2, 628],
 [3, 6],
 [3, 27],
 [3, 64],
 [3, 89],
 [3, 99],
 [3, 100],
 [3, 102],
 [3, 106],
 [3, 127],
 [3, 135],
 [3, 183],
 [3, 194],
 [3, 197],
 [3, 200],
 [3, 235],
 [3, 262],
 [3, 269],
 [3, 285],
 [3, 313],
 [3, 330],
 [3, 331],
 [3, 334],
 [3, 341],
 [3, 349],
 [3, 361],
 [3, 373],
 [3, 385],
 [3, 408],
 [3, 456],
 [3, 485],
 [3, 499],
 [3, 506],
 [3, 529],
 [3, 581],
 [3, 584],
 [3, 591],
 [3, 593],
 [3, 605],
 [3, 621],
 [3, 643],
 [3, 647],
 [3, 655],
 [3, 661],
 [3, 668],
 [3, 672],
 [3, 675],
 [3

In [2]:
from random import sample

In [3]:
k = 2
init_medoids = sample(index_list,k)
init_medoids

kmed = kmedoids(
    matrix, initial_index_medoids=init_medoids, data_type="distance_matrix"
)

kmed.process()

medoid_idxs = kmed.get_medoids()
medoid_idxs

NameError: name 'index_list' is not defined